In [1]:
import import_ipynb
import sys
sys.path.append('C:\\Users\\USER\\JupyterProjects\bilstm_attention_ti_cor')
from Stock_Dataset import StockDataset

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import argparse
from Att_BILSTM import attLSTM
import numpy as np
import time
from metric import metric_acc as ACC
import matplotlib.pyplot as plt
import csv
import os
from loss_fn1 import Selective_Regularization
from Stock_dataloader_csv_ti import stock_csv_read
from Transformer_Encoder import Transformer
from sklearn.metrics import accuracy_score


def train(attLSTM,transformer, lstm_optimizer, transformer_optimizer,trainset , args): ## Data, loss function, argument

    # train dataset을 Dataloader trainset은 주식 개수만큼 append된 list임
    tl =[]
    for i in range(len(trainset)):
        i_trainset = StockDataset(trainset[i][args.split_n])
        trainloader = DataLoader(i_trainset, batch_size = args.batch_size,shuffle=False, drop_last=True)
        tl.append(trainloader)
        
    
    n_data_bundle = []
    for n_data, n_loader in enumerate(tl):
        xy = []
        i_th_batch = []
        for i, (x,y) in enumerate(n_loader):
            # print(x.size()) # torch.Size([64, 10, 17])
            xy.append(x) 
            xy.append(y)
            i_th_batch.append(xy)
        n_data_bundle.append(i_th_batch)
            
    learning_num = len(tl[0])
    attLSTM.train()
    transformer.train()     # train단계라는 것을 명시적으로 알려줌, dropout 같은 단계를 자동으로 적용
    train_loss = 0.0
    for i_th_learning in range(learning_num):
        lstm_optimizer.zero_grad()
        transformer_optimizer.zero_grad()
        for i, data in enumerate(n_data_bundle):
            x = data[i_th_learning][0]
            y = data[i_th_learning][1]
            
            x = x.to(args.device)       
            y = y.squeeze().float().to(args.device)
            
            attLSTM.hidden = [hidden.to(args.device) for hidden in attLSTM.init_hidden()]
            yhat, attention_weight, attn_applied = attLSTM(x)
            
            yhat = yhat.unsqueeze(2)
            y = y.unsqueeze(1)
            if i == 0:
                # torch.Size([64])torch.Size([64, 64])
                Transformer_input = yhat
                True_y = y
            else:
                Transformer_input = torch.cat((Transformer_input, yhat),dim=2)
                True_y = torch.cat((True_y, y),dim=1)
                
        # print(Transformer_input.size()) torch.Size([64, 10, 50])
        output = transformer(Transformer_input.transpose(0,1))

        loss = args.loss_fn(output, True_y)
        loss.backward()
        lstm_optimizer.step()    ## parameter 갱신
        transformer_optimizer.step()
        train_loss += loss.item()
    
    train_loss = train_loss / len(trainloader)
    
    del tl
    del n_data_bundle
    return attLSTM,transformer, train_loss


def validation(attLSTM,transformer, valset, args):

    # train dataset을 Dataloader
    tl =[]
    for i in range(len(valset)):
        i_valset = StockDataset(valset[i][args.split_n])
        valloader = DataLoader(i_valset, batch_size = args.batch_size,shuffle=False, drop_last=True)
        tl.append(valloader)
        
    n_data_bundle = []
    for n_data, n_loader in enumerate(tl):
        
        xy = []
        i_th_batch = []
        for i, (x,y) in enumerate(n_loader):  
            xy.append(x)
            xy.append(y)
            i_th_batch.append(xy)
        n_data_bundle.append(i_th_batch)

    attLSTM.eval()
    transformer.eval()
    val_loss = 0.0
    learning_num = len(tl[0]) # 16
    with torch.no_grad():
        for i_th_learning in range(learning_num):
            for i, data in enumerate(n_data_bundle):
                
                x = data[i_th_learning][0]
                y = data[i_th_learning][1]
                
                x = x.to(args.device)               
                y = y.squeeze().float().to(args.device)

                attLSTM.hidden = [attLSTM.to(args.device) for hidden in attLSTM.init_hidden()]
                yhat, attention_weight, attn_applied = attLSTM(x)
                
                yhat = yhat.unsqueeze(2)
                y = y.unsqueeze(1)
                if i == 0:
                    # torch.Size([64])torch.Size([64, 64])
                    Transformer_input = yhat
                    True_y = y
                else:
                    Transformer_input = torch.cat((Transformer_input, yhat),dim=2)
                    True_y = torch.cat((True_y, y),dim=1)
                    
            output= transformer(Transformer_input.transpose(0,1))
            
            loss = args.loss_fn(output, True_y)
            val_loss += loss.item()

    val_loss = val_loss / len(valloader)
    
    del tl
    del n_data_bundle
    
    return attLSTM, transformer, val_loss


def test(attLSTM, transformer, testset, args):
    # train dataset을 Dataloader
    tl =[]
    for i in range(len(testset)):
        i_testset = StockDataset(testset[i][args.split_n])
        testloader = DataLoader(i_testset, batch_size = args.batch_size,shuffle=False, drop_last=True)
        tl.append(testloader)
        
    n_data_bundle = []
    for n_data, n_loader in enumerate(tl):
        xy = []
        i_th_batch = []
        for i, (x,y) in enumerate(n_loader): 

            xy.append(x)
            xy.append(y)
            i_th_batch.append(xy)
        n_data_bundle.append(i_th_batch)
    
    attLSTM.eval()
    transformer.eval()
    ACC_metric = 0.0
    learning_num = len(tl[0])
    with torch.no_grad():
        for i_th_learning in range(learning_num):
            for i,data in enumerate(n_data_bundle):
                x = data[i_th_learning][0]
                y = data[i_th_learning][1]
                # feature transform
                y = y.squeeze().float().to(args.device)
                x = x.to(args.device)

                attLSTM.hidden = [hidden.to(args.device) for hidden in attLSTM.init_hidden()]

                yhat, attention_weight, attn_applied = attLSTM(x)
                # yhat [64,10]
                yhat = yhat.unsqueeze(2)
                y = y.unsqueeze(1)
                if i == 0:
                    # torch.Size([64])torch.Size([64, 64])
                    Transformer_input = yhat
                    True_y = y
                else:
                    Transformer_input = torch.cat((Transformer_input, yhat),dim=2)
                    True_y = torch.cat((True_y, y),dim=1)

            output= transformer(Transformer_input.transpose(0,1))
            output_ = torch.where(output >= 0.5, 1.0, 0.0)
            output_.requires_grad = True
            
            perc_y_pred = output_.detach().cpu().numpy()
            perc_y_true = True_y.detach().cpu().numpy()
            
            perc_y_predarr = np.array(perc_y_pred)
            perc_y_truearr = np.array(perc_y_true)
            
            acclist_0 = []               
            acclist_1 = []
            acclist_2 = []
            acclist_3 = []
            acclist_4 = []
            acclist_5 = []
            acclist_6 = []
            acclist_7 = []
            acclist_8 = []
            acclist_9 = []
            acclist_10= []
            acclist_11= []
            acclist_12= []
            acclist_13= []
            acclist_14= []
            acclist_15= []
            acclist_16= []
            acclist_17= []
            acclist_18= []
            acclist_19= []
            acclist_20= []
            acclist_21= []
            acclist_22= []
            acclist_23= []
            acclist_24= []
            acclist_25= []
            acclist_26= []
            acclist_27= []
            acclist_28= []
            acclist_29= []
            acclist_30= []
            acclist_31= []
            acclist_32= []
            acclist_33= []
            acclist_34= []
            acclist_35= []
            acclist_36= []
            acclist_37= []
            acclist_38= []
            acclist_39= []
            acclist_40= []
            acclist_41= []
            acclist_42= []
            acclist_43= []
            acclist_44= []
            acclist_45= []
            acclist_46= []
            acclist_47= []
            acclist_48= []
            acclist_49= []
            
            for i in range(50):
                if perc_y_predarr[:,i] != 0:
                    globals()['acc_'+str(i)] = accuracy_score(perc_y_predarr[:,i], perc_y_truearr[:,i])
                else:
                    globals()['acc_'+str(i)] = 0.0
                    
            acclist_0.append(acc_0)                   
            acclist_1.append(acc_1)
            acclist_2.append(acc_2)
            acclist_3.append(acc_3)
            acclist_4.append(acc_4)
            acclist_5.append(acc_5)
            acclist_6.append(acc_6)
            acclist_7.append(acc_7)
            acclist_8.append(acc_8)
            acclist_9.append(acc_9)
            acclist_10.append(acc_10)
            acclist_11.append(acc_11)
            acclist_12.append(acc_12)
            acclist_13.append(acc_13)
            acclist_14.append(acc_14)
            acclist_15.append(acc_15)
            acclist_16.append(acc_16)
            acclist_17.append(acc_17)
            acclist_18.append(acc_18)
            acclist_19.append(acc_19)
            acclist_20.append(acc_20)
            acclist_21.append(acc_21)
            acclist_22.append(acc_22)
            acclist_23.append(acc_23)
            acclist_24.append(acc_24)
            acclist_25.append(acc_25)
            acclist_26.append(acc_26)
            acclist_27.append(acc_27)
            acclist_28.append(acc_28)
            acclist_29.append(acc_29)
            acclist_30.append(acc_30)
            acclist_31.append(acc_31)
            acclist_32.append(acc_32)
            acclist_33.append(acc_33)
            acclist_34.append(acc_34)
            acclist_35.append(acc_35)
            acclist_36.append(acc_36)
            acclist_37.append(acc_37)
            acclist_38.append(acc_38)
            acclist_39.append(acc_39)
            acclist_40.append(acc_40)
            acclist_41.append(acc_41)
            acclist_42.append(acc_42)
            acclist_43.append(acc_43)
            acclist_44.append(acc_44)
            acclist_45.append(acc_45)
            acclist_46.append(acc_46)
            acclist_47.append(acc_47)
            acclist_48.append(acc_48)
            acclist_49.append(acc_49)

            ## acc list가 데이터 개수만큼 데이터 순서대로 01234... 생김
                    
            #ACC_metric += acc
            #ACC_metric += ACC(perc_y_pred, perc_y_true) # numpy.float64' object is not callable
    acc_metric_list = []
    for ACC_metric in globals():
        if "ACC_metric" in ACC_metric:
            ACC_metric = ACC_metric / len(testloader)
            acc_metric_list.append(ACC_metric)
    return acc_metric_list




In [3]:

# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

# ========= experiment setting ========== #
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

args.save_file_path = "C:\\Users\\USER\\JupyterProjects\\bilstm_attention_ti_cor\\results"

# ====== hyperparameter ======= #
args.batch_size = 64

args.dropout = 0.2
args.use_bn = True
args.loss_fn = nn.BCELoss()  ## loss function for classification : cross entropy
args.optim = 'Adam'
args.lr = 0.0005
args.l2 = 0.00001 #?
args.epoch = 100

args.ent_split_n = 10
# ============= model ================== #
args.attLSTM = attLSTM
args.transformer = Transformer
# ====== att_lstm hyperparameter ======= #
args.x_frames = 10
args.y_frames = 1

args.input_dim = 64
args.hid_dim = 64
args.output_dim = 1

args.attention_head = 1
args.attn_size = 10
args.num_layers = 1
args.attLSTM_x_frames = 1


# ====== transformer hyperparameter ======= #
args.trans_feature_size = 5
args.trans_num_laysers = 1
args.trans_nhead = 1
args.market_beta = 0.1

# trans_feature_size / trans_nhead => int 필수


In [4]:
## 실행 파일
file_tf=os.path.isdir(r"C:\Users\USER\JupyterProjects\bilstm_attention_ti_cor\results\0_split")
if file_tf == True:
    sys.exit("file is already exist")

with open(args.save_file_path + '\\' + 'ATTBILSTM_result_t.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["model", "stock", "entire_exp_time",  "avg_test_ACC"])
    
    # stock = data.split('.')[0]

    est = time.time()
    # setattr(args, 'symbol', stock)
    # args.new_file_path = args.save_file_path + '\\' + "ATTBILSTM_" + args.symbol
    # os.makedirs(args.new_file_path)
        
    csv_read = stock_csv_read(args.x_frames,args.y_frames)
    stock_50_data, data_list = csv_read.cv_split() ## 분할된 50개의 주식, data_list를 한번에 뱉어줌
    
    ACC_cv = []
    for split_n, i_split in enumerate(range(args.ent_split_n)):
        args.split_n = split_n
        
        args.split_file_path = args.save_file_path + '\\' + str(args.split_n) + "_split"
        os.makedirs(args.split_file_path)
        
        attLSTM = args.attLSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,args.dropout, args.use_bn, args.attention_head, args.attn_size,activation="ReLU")
        transformer = args.transformer(args.trans_feature_size, args.trans_num_laysers, args.dropout, args.batch_size, args.x_frames, args.trans_nhead)
    
        attLSTM.to(args.device)
        transformer.to(args.device)
        
        if args.optim == 'SGD':
            lstm_optimizer = optim.SGD(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
            transformer_optimizer = optim.SGD(transformer.parameters(), lr=args.lr, weight_decay=args.l2)

        elif args.optim == 'RMSprop':
            lstm_optimizer = optim.RMSprop(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
            transformer_optimizer = optim.RMSprop(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
        elif args.optim == 'Adam':
            lstm_optimizer = optim.Adam(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
            transformer_optimizer = optim.Adam(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
        else:
            raise ValueError('In-valid optimizer choice')

        # ===== List for epoch-wise data ====== #
        train_losses = []
        val_losses = []
        # ===================================== #
        for epoch in range(args.epoch):
            ts = time.time()
            attLSTM, transformer, train_loss = train(attLSTM, transformer, lstm_optimizer, transformer_optimizer, stock_50_data[0], args)
            attLSTM, transformer, val_loss = validation(attLSTM, transformer, stock_50_data[1], args)

            te = time.time()

            ## 각 에폭마다 모델을 저장하기 위한 코드
            if len(val_losses) == 0:
                torch.save(attLSTM.state_dict(), args.split_file_path + '\\' + str(epoch) +'_attLSTM' +'.pt')
                torch.save(transformer.state_dict(), args.split_file_path + '\\' + str(epoch) +'_transformer' +'.pt')
            elif min(val_losses) > val_loss:
                torch.save(attLSTM.state_dict(), args.split_file_path + '\\' + str(epoch) +'_attLSTM' +'.pt')
                torch.save(transformer.state_dict(), args.split_file_path + '\\' + str(epoch) +'_transformer' +'.pt')

            train_losses.append(train_loss)
            val_losses.append(val_loss)

            print('Epoch {}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
                    .format(epoch, train_loss, val_loss, te - ts))

        ## val_losses에서 가장 값이 최소인 위치를 저장함
        site_val_losses = val_losses.index(min(val_losses)) ## 10 epoch일 경우 0번째~9번째 까지로 나옴
        attLSTM = args.attLSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                                            args.dropout, args.use_bn, args.attention_head, args.attn_size, activation="ReLU")
        transformer = args.transformer(args.trans_feature_size, args.trans_num_laysers,args.dropout, args.batch_size, args.x_frames, args.trans_nhead)

        attLSTM.to(args.device)
        transformer.to(args.device)
        
        attLSTM.load_state_dict(torch.load(args.split_file_path + '\\' + str(site_val_losses) +'_attLSTM'+ '.pt'))
        transformer.load_state_dict(torch.load(args.split_file_path + '\\' + str(site_val_losses) +'_transformer' + '.pt'))

        ACC = test(attLSTM, transformer, stock_50_data[2], args)
        print(ACC)
        print('ACC: {}'.format(ACC))

        with open(args.split_file_path + '\\'+ str(site_val_losses)+'Epoch_test_metric' +'.csv', 'w') as fd:
            print('ACC: {}'.format(ACC), file=fd)

        result = {}

        result['train_losses'] = train_losses
        result['val_losses'] = val_losses
        result['ACC'] = ACC
    
    # =============================================================================================================================================== #
        eet = time.time()
        entire_exp_time = eet - est

        fig = plt.figure()
        plt.plot(result['train_losses'])
        plt.plot(result['val_losses'])
        plt.legend(['train_losses', 'val_losses'], fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('loss', fontsize=15)
        plt.grid()
        plt.savefig(args.split_file_path + '\\' + 'fig' + '.png')
        plt.close(fig)
        ACC_cv.append(result['ACC'])
        
    ACC_cv_ar = np.array(ACC_cv)
    acc_avg = np.mean(ACC_cv_ar)
    acc_std = np.std(ACC_cv_ar)

    wr.writerow(["BILSTM_ATTENTION", args.symbol, entire_exp_time, acc_avg, acc_std])

Epoch 0, Loss(train/val) 0.69550/0.69935. Took 0.70 sec
Epoch 1, Loss(train/val) 0.67839/0.69800. Took 0.55 sec
Epoch 2, Loss(train/val) 0.64948/0.71707. Took 0.46 sec
Epoch 3, Loss(train/val) 0.58003/0.75956. Took 0.47 sec
Epoch 4, Loss(train/val) 0.52053/0.80875. Took 0.46 sec
Epoch 5, Loss(train/val) 0.47464/0.86004. Took 0.46 sec
Epoch 6, Loss(train/val) 0.43007/0.92672. Took 0.44 sec
Epoch 7, Loss(train/val) 0.39932/0.98364. Took 0.45 sec
Epoch 8, Loss(train/val) 0.36869/1.03576. Took 0.45 sec
Epoch 9, Loss(train/val) 0.34147/1.11022. Took 0.47 sec
Epoch 10, Loss(train/val) 0.31730/1.13921. Took 0.49 sec
Epoch 11, Loss(train/val) 0.29802/1.18482. Took 0.45 sec
Epoch 12, Loss(train/val) 0.27935/1.23398. Took 0.44 sec
Epoch 13, Loss(train/val) 0.26747/1.26959. Took 0.43 sec
Epoch 14, Loss(train/val) 0.25679/1.31356. Took 0.44 sec
Epoch 15, Loss(train/val) 0.24750/1.38056. Took 0.50 sec
Epoch 16, Loss(train/val) 0.24311/1.39976. Took 0.44 sec
Epoch 17, Loss(train/val) 0.23499/1.46313

AttributeError: 'str' object has no attribute 'append'